In [1]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [3]:
class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/search/ajax/?' +\
        'project={0}&'.format(param_dict['project']) +\
        'category={0}&'.format(param_dict['category']) +\
        'dateFrom={0}&'.format(param_dict['dateFrom']) +\
        'dateTo={0}&'.format(param_dict['dateTo']) +\
        'page={0}&'.format(param_dict['page']) +\
        'query={0}&'.format(param_dict['query']) +\
        'material={0}'.format(param_dict['material'])
        
        # 'offset={0}&'.format(param_dict['offset']) +\
        # 'limit={0}&'.format(param_dict['limit']) +\
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          include_text: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        include_text: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if include_text and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
        
        if 'publish_date_t' in search_table.columns:
            search_table.sort_values('publish_date_t', ignore_index=True)
            
        return search_table
    
    
    def _iterable_load_by_page(self, param_dict):
        param_copy = param_dict.copy()
        results = []
        
        result = self._get_search_table(param_copy)
        results.append(result)
        
        while not result.empty:
            param_copy['page'] = str(int(param_copy['page']) + 1)
            result = self._get_search_table(param_copy)
            results.append(result)
            
        results = pd.concat(results, axis=0, ignore_index=True)
        
        return results
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### query - поисковой запрос (ключевое слово), например, РБК
        ###### page - смещение поисковой выдачи (с шагом 20)
        
        ###### Deprecated:
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._iterable_load_by_page(param_copy)], axis=0, ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
        
        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

In [4]:
# Задаем параметры запроса и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2024-10-29'
dateTo = "2024-10-30"
page = 0

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'page'   : str(page),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '29.10.2024', 'dateTo': '30.10.2024', 'page': '0', 'material': ''}


In [5]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 20 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               include_text = True) # Парсить текст статей
print(len(tbl))
tbl.head(20)

20


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6721d7ee9a79471dad034abd,РБК,rbcnews,short_news,Финансы,«Известия» напомнили о налоге на доходы от вкл...,,2024-10-30T10:16:41+03:00,1730272601,https://www.rbc.ru/rbcfreenews/6721d7ee9a79471...,https://s0.rbk.ru/v6_top_pics/media/img/1/75/3...,None,free,None,1,None,Под налог по итогам 2024 года попадут доходы о...
1,6721c7b59a7947f5e247c505,РБК,rbcnews,article,Спорт,Овечкин впервые в сезоне сделал дубль и сократ...,Российский форвард Александр Овечкин впервые в...,2024-10-30T10:07:19+03:00,1730272039,https://www.rbc.ru/sport/30/10/2024/6721c7b59a...,https://s0.rbk.ru/v6_top_pics/media/img/2/12/3...,None,free,None,1,Российский форвард Александр Овечкин впервые в...,«Вашингтон Кэпиталз» обыграл «Нью-Йорк Рейндже...
2,6721ce5e9a79470242232f14,РБК,rbcnews,article,Политика,Харрис назвала целью Трампа создание «большего...,Харрис назвала Трампа «неуравновешенным» и ука...,2024-10-30T10:02:20+03:00,1730271740,https://www.rbc.ru/politics/30/10/2024/6721ce5...,https://s0.rbk.ru/v6_top_pics/media/img/4/12/3...,None,free,None,1,Харрис назвала Трампа «неуравновешенным» и ука...,Республиканец Дональд Трамп стремится к бескон...
3,6721d2c39a7947210be19006,РБК,rbcnews,short_news,Политика,Посольству России не сообщили о задержании жур...,,2024-10-30T09:56:25+03:00,1730271385,https://www.rbc.ru/rbcfreenews/6721d2c39a79472...,https://s0.rbk.ru/v6_top_pics/media/img/0/38/3...,None,free,None,1,None,Власти США не сообщили в посольство России в В...
4,6721cc239a79475d15fe6523,РБК,rbcnews,article,Политика,"NYT узнала, что Израиль нанес удар по крупному...",В ночь на 26 октября Израиль нанес авиаудары п...,2024-10-30T09:43:23+03:00,1730270603,https://www.rbc.ru/politics/30/10/2024/6721cc2...,https://s0.rbk.ru/v6_top_pics/media/img/8/94/3...,None,free,None,1,В ночь на 26 октября Израиль нанес авиаудары п...,В результате атаки израильских ВВС на Иран в н...
5,6721d2a09a79479e2d706e56,РБК,rbcnews,article,Общество,Шереметьево на час ограничивало полеты,Ухода рейсов на запасные аэродромы не было,2024-10-30T09:31:00+03:00,1730269860,https://www.rbc.ru/society/30/10/2024/6721d2a0...,https://s0.rbk.ru/v6_top_pics/media/img/2/47/3...,None,free,None,1,Ухода рейсов на запасные аэродромы не было,Столичный аэропорт Шереметьево не принимал рей...
6,6721cc2f9a79470e3835e239,РБК,rbcnews,short_news,Общество,"В Москве выявили схемы, по которым более 100 ч...",,2024-10-30T09:30:21+03:00,1730269821,https://www.rbc.ru/rbcfreenews/6721cc2f9a79470...,https://s0.rbk.ru/v6_top_pics/media/img/3/12/3...,None,free,None,1,None,В Москве возбудили уголовное дело против мошен...
7,6720d63d9a79478690f58b0b,РБК,rbcnews,article,Экономика,В РАНХиГС сообщили о снижении доли государства...,"Доля государства в ВВП сократилась с 51,7 до 4...",2024-10-30T09:30:19+03:00,1730269819,https://www.rbc.ru/economics/30/10/2024/6720d6...,https://s0.rbk.ru/v6_top_pics/media/img/7/11/3...,None,free,None,1,"Доля государства в ВВП сократилась с 51,7 до 4...",Доля госсектора в экономике России в 2023 году...
8,6720fff19a794704eedc8a70,РБК,rbcnews,article,Спорт,Экс-судья КХЛ выступил за дисквалификацию трен...,КХЛ оштрафовала «Спартак» из-за поведения боле...,2024-10-30T09:14:08+03:00,1730268848,https://www.rbc.ru/sport/30/10/2024/6720fff19a...,https://s0.rbk.ru/v6_top_pics/media/img/7/00/3...,None,free,None,1,КХЛ оштрафовала «Спартак» из-за поведения боле...,Главного тренера петербургского СКА Романа Рот...
9,6721c5499a79470d23d6121b,РБК,rbcnews,article,Политика,Microsoft выявила атаку хакеров из «Полуночной...,Microsoft обвинила хакерскую группировку Midni...,2024-10-30T09:08:31+03:00,1730268511,https://www.rbc.ru/politics/30/10/2024/6721c54...,https://s0.rbk.ru/v6_top_pics/media/img/4/07/3...,None,free,None,1,Microsoft обвинила хакерскую группировку Midni...,Российская хакерская группировка Midnight Bliz...


In [6]:
# Сохранение данных в CSV
tbl.to_csv('articles.csv', index=False)

# Сохранение данных в JSON
tbl.to_json('articles.json', orient='records', lines=True)

In [7]:
%%time
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict=param_dict,
                             time_step = 1, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                             save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Parsing articles from 29.10.2024 to 30.10.2024
Finish
241
CPU times: total: 3.42 s
Wall time: 47.4 s


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6721d7ee9a79471dad034abd,РБК,rbcnews,short_news,Финансы,«Известия» напомнили о налоге на доходы от вкл...,,2024-10-30T10:16:41+03:00,1730272601,https://www.rbc.ru/rbcfreenews/6721d7ee9a79471...,https://s0.rbk.ru/v6_top_pics/media/img/1/75/3...,None,free,None,1,None,Под налог по итогам 2024 года попадут доходы о...
1,6721c7b59a7947f5e247c505,РБК,rbcnews,article,Спорт,Овечкин впервые в сезоне сделал дубль и сократ...,Российский форвард Александр Овечкин впервые в...,2024-10-30T10:07:19+03:00,1730272039,https://www.rbc.ru/sport/30/10/2024/6721c7b59a...,https://s0.rbk.ru/v6_top_pics/media/img/2/12/3...,None,free,None,1,Российский форвард Александр Овечкин впервые в...,«Вашингтон Кэпиталз» обыграл «Нью-Йорк Рейндже...
2,6721ce5e9a79470242232f14,РБК,rbcnews,article,Политика,Харрис назвала целью Трампа создание «большего...,Харрис назвала Трампа «неуравновешенным» и ука...,2024-10-30T10:02:20+03:00,1730271740,https://www.rbc.ru/politics/30/10/2024/6721ce5...,https://s0.rbk.ru/v6_top_pics/media/img/4/12/3...,None,free,None,1,Харрис назвала Трампа «неуравновешенным» и ука...,Республиканец Дональд Трамп стремится к бескон...
3,6721d2c39a7947210be19006,РБК,rbcnews,short_news,Политика,Посольству России не сообщили о задержании жур...,,2024-10-30T09:56:25+03:00,1730271385,https://www.rbc.ru/rbcfreenews/6721d2c39a79472...,https://s0.rbk.ru/v6_top_pics/media/img/0/38/3...,None,free,None,1,None,Власти США не сообщили в посольство России в В...
4,6721cc239a79475d15fe6523,РБК,rbcnews,article,Политика,"NYT узнала, что Израиль нанес удар по крупному...",В ночь на 26 октября Израиль нанес авиаудары п...,2024-10-30T09:43:23+03:00,1730270603,https://www.rbc.ru/politics/30/10/2024/6721cc2...,https://s0.rbk.ru/v6_top_pics/media/img/8/94/3...,None,free,None,1,В ночь на 26 октября Израиль нанес авиаудары п...,В результате атаки израильских ВВС на Иран в н...
